# Periodic boundary conditions

boundary value problem on $\textbf{x}\in\Omega$

$$\nabla^2 u + u = f(\textbf{x})$$

Neumann boundary condition on $\textbf{x}\in\partial\Omega_{\text{D}}$

$$u = u_{\text{D}}$$

Neumann boundary condition on $\textbf{x}\in\partial\Omega_{\text{N}}$ 

$$\textbf{n}\cdot\nabla u = u_{\text{N}}$$

periodic boundary condition on $\textbf{x}\in\partial\Omega_{\text{M}}\cup\in\partial\Omega_{\text{S}}$

$$u\vert_{\partial\Omega_{\text{M}}} = u\vert_{\partial\Omega_{\text{S}}}$$

In [ ]:
from ufl.core.expr import Expr
from ufl import Form, inner, grad, dx, TestFunction, TrialFunction
from dolfinx.fem import FunctionSpace

from lucifex.mesh import rectangle_mesh, mesh_boundary
from lucifex.fem import LUCiFExFunction as Function, LUCiFExConstant as Constant
from lucifex.solver import bvp_solver, BoundaryConditions
from lucifex.viz import plot_colormap, plot_line
from lucifex.utils import cross_section


def forced_helmholtz(
    u: Function,
    f: Function | Constant | Expr,
) -> tuple[Form, Form]:
    v = TestFunction(u.function_space)
    u_trial = TrialFunction(u.function_space)
    F_lhs = -inner(grad(v), grad(u_trial)) * dx
    F_lhs += v * u * dx
    F_rhs = -v * f * dx
    return F_lhs, F_rhs

### $d=2~,~\partial\Omega_{\text{D}}\cup\partial\Omega_{\text{N}}\neq\varnothing$

$$\Omega = [0, L_x] \times [0, L_y]$$

$$c(x=0,y)=c(x=L_x,y)$$

$$c(x,y=0)=0$$

$$\frac{\partial c}{\partial x}\bigg\vert_{x,y=L_y}=0$$

In [ ]:
Lx = 2.0
Ly = 1.0
mesh = rectangle_mesh(Lx, Ly, 20, 10)
boundary = mesh_boundary(
    mesh, 
    {
        "left": lambda x: x[0],
        "right": lambda x: x[0] - Lx,
        "lower": lambda x: x[1],
        "upper": lambda x: x[1] - Ly,
    },
)

bcs = BoundaryConditions(
    ("dirichlet", boundary['lower'], 0.0),
    ("neumann", boundary['upper'], 0.0), 
    ('periodic', boundary['left'], boundary['right']) ,
)

### $d=2~,~\partial\Omega_{\text{D}}\cup\partial\Omega_{\text{N}}=\varnothing$

$$\Omega = [0, L_x] \times [0, L_y]$$

$$c(x=0,y)=c(x=L_x,y)$$

$$c(x,y=0)=c(x,y=L_y)$$

In [ ]:
bcs = BoundaryConditions(
    ('periodic', boundary['lower'], boundary['upper']),
    ('periodic', boundary['left'], boundary['right']),
)